In [108]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import re

In [70]:
train = pd.read_csv('./datasets/titanic/train.csv')
test = pd.read_csv('./datasets/titanic/test.csv')
num_train = len(train)

In [71]:
df_all = pd.concat([train, test], sort=False)

In [72]:
df_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
df_all['Embarked'] = df_all['Embarked'].fillna('S')

In [74]:
df_all = df_all.drop(columns=['Ticket', 'Cabin'])

Sex

In [75]:
df_all['Sex'] = df_all['Sex'].map({'female': 0, 'male': 1}).astype(int)

age proccesing 

In [76]:
for i in range(0, 2):
    for j in range(0, 3):
        df_all.loc[(df_all['Age'].isna()) & (df_all['Sex'] == i) & (df_all['Pclass'] == j + 1), 'Age'] = df_all[(df_all['Sex'] == i) & (df_all['Pclass'] == j + 1)]['Age'].dropna().median()

In [77]:
df_all.loc[ df_all['Age'] <= 16, 'Age'] = 0
df_all.loc[(df_all['Age'] > 16) & (df_all['Age'] <= 32), 'Age'] = 1
df_all.loc[(df_all['Age'] > 32) & (df_all['Age'] <= 48), 'Age'] = 2
df_all.loc[(df_all['Age'] > 48) & (df_all['Age'] <= 64), 'Age'] = 3
df_all.loc[ df_all['Age'] > 64, 'Age'] = 4

fare processing

In [80]:
df_all.loc[ df_all['Fare'] <= 7.91, 'Fare'] = 0
df_all.loc[(df_all['Fare'] > 7.91) & (df_all['Fare'] <= 14.454), 'Fare'] = 1
df_all.loc[(df_all['Fare'] > 14.454) & (df_all['Fare'] <= 31), 'Fare']  = 2
df_all.loc[ df_all['Fare'] > 31, 'Fare'] = 3

In [81]:
df_all['Fare'].fillna(df_all['Fare'].dropna().median(), inplace=True)

Is alone processing

In [82]:
df_all['IsAlone'] = 0
df_all.loc[df_all['SibSp'] + df_all['Parch'] == 0, 'IsAlone'] = 1

In [83]:
df_all = df_all.drop(columns=['SibSp', 'Parch'])

Title

In [84]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

In [85]:
df_all['Name'] = df_all['Name'].apply(get_title)
df_all['Name'] = df_all['Name'].replace(['Lady', 'Countess','Capt', 'Col',\
'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df_all['Name'] = df_all['Name'].replace('Mlle', 'Miss')
df_all['Name'] = df_all['Name'].replace('Ms', 'Miss')
df_all['Name'] = df_all['Name'].replace('Mme', 'Mrs')
        

In [86]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
df_all['Name'] = df_all['Name'].map(title_mapping)
df_all['Name'] = df_all['Name'].fillna(0)

Embarked

In [87]:
df_all['Embarked'] = df_all['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

Age*class

In [88]:
df_all['Age*class'] = df_all['Age'] * df_all['Pclass']

In [89]:
df_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Fare,Embarked,IsAlone,Age*class
0,1,0.0,3,1,1,1.0,0.0,0,0,3.0
1,2,1.0,1,3,0,2.0,3.0,1,0,2.0
2,3,1.0,3,2,0,1.0,1.0,0,1,3.0
3,4,1.0,1,3,0,2.0,3.0,0,0,2.0
4,5,0.0,3,1,1,2.0,1.0,0,1,6.0


Split df on test and train set's

In [90]:
X_train = df_all.iloc[:num_train].drop(columns=['Survived', 'PassengerId'])
y_train = df_all.iloc[:num_train]['Survived']

In [91]:
y_train = pd.get_dummies(y_train)

In [92]:
y_train.head()

,0.0,1.0
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0


In [93]:
X_test = df_all.iloc[num_train:].drop(columns=['Survived'])

In [94]:
y_train.head()

,0.0,1.0
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0


In [95]:
ID = X_test['PassengerId']
X_test = X_test.drop(columns=['PassengerId'])

In [96]:
X_test.head()

,Pclass,Name,Sex,Age,Fare,Embarked,IsAlone,Age*class
0,3,1,1,2.0,0.0,2,1,6.0
1,3,3,0,2.0,0.0,0,0,6.0
2,2,1,1,3.0,1.0,2,1,6.0
3,3,1,1,1.0,1.0,0,1,3.0
4,3,3,0,1.0,1.0,0,0,3.0


# training

In [114]:
def create_model(layers=1, neurons=20, dropout=0.2, activation='relu'):
    model = Sequential()
    model.add(InputLayer(input_shape=(X_train.shape[1],)))
    
    for _ in range(layers):
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(dropout))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

In [129]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [136]:
gsc = GridSearchCV(
    estimator=model,
    param_grid={
        'neurons':[8, 16, 32, 64, 128],
        'layers': range(1, 4),
        'epochs': [40, 60, 100, 150],
        #'dropout': [0, 0.2],
        #'activation': ['tanh', 'relu']
    },
    #scoring='accuracy',
    cv=3
)

In [ ]:
grid_result = gsc.fit(X_train, y_train, verbose=2)

In [ ]:
"Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)

In [ ]:
for test_mean, test_stdev, train_mean, train_stdev, param in zip(
        grid_result.cv_results_['mean_test_score'],
        grid_result.cv_results_['std_test_score'],
        grid_result.cv_results_['mean_train_score'],
        grid_result.cv_results_['std_train_score'],
        grid_result.cv_results_['params']):
    print("Train: %f (%f) // Test : %f (%f) with: %r" % (train_mean, train_stdev, test_mean, test_stdev, param))

In [140]:
model = create_model(3, 64)
model.fit(X_train, y_train, validation_split=0.2, epochs=100, verbose=1)

Train on 712 samples, validate on 179 samples
Epoch 1/100
712/712 [==============================] - 33s 47ms/step - loss: 0.6323 - acc: 0.6545 - val_loss: 0.4675 - val_acc: 0.8380
Epoch 2/100
712/712 [==============================] - 1s 819us/step - loss: 0.5464 - acc: 0.7303 - val_loss: 0.4317 - val_acc: 0.8156
Epoch 3/100
712/712 [==============================] - 1s 844us/step - loss: 0.5434 - acc: 0.7669 - val_loss: 0.4816 - val_acc: 0.7877
Epoch 4/100
712/712 [==============================] - 1s 879us/step - loss: 0.5164 - acc: 0.7598 - val_loss: 0.4459 - val_acc: 0.7989
Epoch 5/100
712/712 [==============================] - 1s 867us/step - loss: 0.5034 - acc: 0.7725 - val_loss: 0.4435 - val_acc: 0.7933
Epoch 6/100
712/712 [==============================] - 1s 820us/step - loss: 0.5057 - acc: 0.7823 - val_loss: 0.4283 - val_acc: 0.8156
Epoch 7/100
712/712 [==============================] - 1s 848us/step - loss: 0.4932 - acc: 0.7851 - val_loss: 0.4002 - val_acc: 0.8380
Epoch 8/1

Epoch 61/100
712/712 [==============================] - 1s 899us/step - loss: 0.4118 - acc: 0.8132 - val_loss: 0.3718 - val_acc: 0.8436
Epoch 62/100
712/712 [==============================] - 1s 918us/step - loss: 0.4118 - acc: 0.8287 - val_loss: 0.3642 - val_acc: 0.8492
Epoch 63/100
712/712 [==============================] - 1s 938us/step - loss: 0.4120 - acc: 0.8160 - val_loss: 0.3838 - val_acc: 0.8715
Epoch 64/100
712/712 [==============================] - 1s 887us/step - loss: 0.4048 - acc: 0.8287 - val_loss: 0.3866 - val_acc: 0.8603
Epoch 65/100
712/712 [==============================] - 1s 877us/step - loss: 0.4034 - acc: 0.8301 - val_loss: 0.3713 - val_acc: 0.8492
Epoch 66/100
712/712 [==============================] - 1s 912us/step - loss: 0.4039 - acc: 0.8258 - val_loss: 0.3721 - val_acc: 0.8492
Epoch 67/100
712/712 [==============================] - 1s 888us/step - loss: 0.3992 - acc: 0.8160 - val_loss: 0.3929 - val_acc: 0.8492
Epoch 68/100
712/712 [==========================

In [141]:
y_test = model.predict_classes(X_test)

In [142]:
df_subm = pd.DataFrame({'PassengerId': ID, 'Survived': y_test})
df_subm.to_csv('titanic_subm.csv', index=False)